# Assessing the Reliability of Water Distribution Systems to Seismic Hazards: A Case Study of Shelby County, TN

## 1- Introduction
This Jupyter notebook investigates the reliability of water distribution systems (WDS) in the face of seismic hazards. Utilizing the WNTR Python package, we simulate earthquake impacts on the WDS's infrastructure, including pipelines, tanks, and pumps, to understand how such events might compromise serviceability. Our goal is to identify vulnerabilities and evaluate the system's ability to meet demand post-earthquake. Through hydraulic simulations informed by seismic data, we offer insights into enhancing the resilience of water distribution networks. 

## 2- Setup

### 2-1- Importing Dependencies

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import expon, lognorm, poisson
import random
import wntr
from geopy.distance import great_circle

### 2-2- Loading and Processing Data
In this notebook, we use the Shelby County WDS as our case study. Our analysis requires two types of data: *water distribution system data* and *seismic data*.

**Water Distribution System Data:**

WDS data consists of various attributes related to the water distribution network and is primarily sourced from the [**IN-CORE platform**](https://incore.ncsa.illinois.edu/).

* **Nodal Attributes:** Demand and geolocation (latitude/longitude) for each network node.
* **Pipeline Attributes:** Diameter and length of all pipelines.
* **Network Layout:** Spatial representation of pipes, nodes, tanks, and pumps.
* **EPANET File:** A pre-configured EPANET file for accurate water flow and pressure simulations.


In [2]:
# Load the coordinates of pipelines, tanks, and pumps from CSV input files for further analysis.
# The coordinates of each pipeline is considered to be the geometric center of the pipeline.
pipelines_coordinates = pd.read_csv('input_files/pipelines_coordinates.csv', index_col='pipe_id')
tanks_coordinates = pd.read_csv('input_files/tanks_coordinates.csv', index_col='tank_id')
pumps_coordinates = pd.read_csv('input_files/pumps_coordinates.csv', index_col='pump_id')

**Seismic Data:**

In the context of WDSs, there are primarily two types of seismic hazards: (1) *transient ground deformation* (TGD), which occurs due to ground shaking, and (2) *permanent ground deformation* (PGD), which arises from fault displacement, landslides, or liquefaction-induced ground failures. In this study, we specifically focus on the TGD hazard. 

The necessary seismic data for TGD hazard assessment includes the following:

* **Magnitude:** The magnitude of the earthquake.
* **Epicenter:** The geographic location (latitude and longitude) of the earthquake's epicenter.
* **Depth:** The depth of the earthquake's focus.

The seismic data for this study is based on the scenario described by [Adachi and Ellingwood (2006)](https://www.sciencedirect.com/science/article/pii/S0951832006002493).

In [3]:
magnitude, epicenter, depth = 7.7, (35.18, -90.18), 10000  # The unit of depth is in m

## 3- Simulation


In this section, we focus on simulating the seismic impacts on the WDS to assess the serviceability of its components. Serviceability is a crucial measure of the system's ability to function effectively post-earthquake. We employ the [**WNTR package**](https://usepa.github.io/WNTR/), a powerful tool for hydraulic simulation, to model the behavior of the damaged WDS under seismic condition.

### 3-1- Setting Up the Water Distribution System Model

The hydraulic demand model in WNTR package is set to Pressure-Dependent Demand (PDD), which adjusts water demand based on the available pressure at each node.

In [4]:
# Define the WDS model by reading an EPANET input file through WNTR package. 
wn = wntr.network.WaterNetworkModel('input_files/Shelby_County_WDN.inp')

# Set network parameters
wn.options.hydraulic.demand_model = 'PDD'    

wn.options.hydraulic.minimum_pressure = 3.52  # 5 psi
wn.options.hydraulic.required_pressure = 14.06 # 20 psi 

wn.options.time.duration = 24 * 3600  # 24 hours
leak_start_time = 0 * 3600
leak_end_time = 24 * 3600

### 3-2- Initializing the Earthquake Object

In this section, we initialize the earthquake object using the WNTR package, which requires specifying the earthquake's magnitude, epicenter (latitude and longitude), and depth. 

We then use WNTR's attenuation models to calculate peak ground velocity (PGV) and peak ground acceleration (PGA) values at the location of each pipeline, tank and pump based on their distance from the earthquake's epicenter. 

The distances are determined using the great-circle function from the Scipy package, which calculates the shortest distance between two points on a sphere's surface based on their longitudes and latitudes, as provided in the input CSV files.

For PGA, the following attenuation models are supported in WNTR package:
- **Kawashima et al. 1984**
- **Baag et al. 1998**
- **Lee and Cho 2002**

For PGV, the **Yu and Jin (2008)** model is used in WNTR package.

In [5]:
earthquake = wntr.scenario.Earthquake(epicenter, magnitude, depth) 

### 3-3- Evaluating Earthquake Effects on Tanks

For tanks, serviceability is determined by the ratio of the water level post-earthquake to the original water level. The reduction in water level can result from physical damage to the tank, determined using fragility curves, or due to decreased system pressure caused by damage to water pipelines.

In this study, we employ fragility curves defined by [**HAZUS-MH**](https://www.fema.gov/sites/default/files/2020-09/fema_hazus_earthquake-model_technical-manual_2.1.pdf) for tanks, which are represented as lognormal distributions with peak ground acceleration (PGA) as the engineering demand parameter. The corresponding median and lognormal standard deviation (β) values are provided in the following table, with units in g:
| Damage State | Median (g) | β   |
|--------------|------------|-----|
| Minor        | 0.18       | 0.5 |
| Moderate     | 0.55       | 0.5 | 
| Extensive    | 1.15       | 0.6 |
| Complete     | 1.50       | 0.6 |

For tanks, capacities are adjusted to 60%, 80%, and 90% for extensive, moderate, and minor damage states, respectively **(El-Tawil and Lin 2019)**.



In [6]:
# Lognormal distribution parameters for each damage state (median, beta) of the tanks
tanks_damage_state_params = {'Minor': (0.18, 0.5), 'Moderate': (0.55, 0.5), 'Extensive': (1.15, 0.6), 'Complete': (1.50, 0.6)}

# Define operational levels for each damage state of the tanks
tank_operational_levels = {'None': 1.0, 'Minor': 0.9, 'Moderate': 0.8, 'Extensive': 0.6, 'Complete': 0.0}

# Function to determine the damage state of a tank based on its PGA value
def determine_tank_damage_state(pga):
    probabilities = {}
    previous_probability = 0
    for state, params in tanks_damage_state_params.items():
        median, beta = params
        exceedance_probability = 1 - lognorm.cdf(pga, s=beta, scale=median)
        probabilities[state] = exceedance_probability - previous_probability
        previous_probability = exceedance_probability

    random_number = random.uniform(0, 1)
    damage_state = 'None'
    cumulative_probability = 0
    for state, probability in probabilities.items():
        cumulative_probability += probability
        if random_number <= cumulative_probability:
            damage_state = state
            break
    return damage_state

# Get the original tank levels
original_tank_levels = {tank_name: wn.get_node(tank_name).init_level for tank_name in wn.tank_name_list}

# Calculate the distance from the epicenter to each tank
tanks_distances = tanks_coordinates.apply(lambda row: great_circle((row['latitude'], row['longitude']), epicenter).meters, axis=1)

# Calculate the pga values at the location of each tank
tanks_pga = earthquake.pga_attenuation_model(tanks_distances)

# Determine the damage state for each tank and adjust its initial water level
for tank_name in wn.tank_name_list:
    tank = wn.get_node(tank_name)
    pga_value = tanks_pga.get(tank_name, 0)  # Get the PGA value for each tank, default to 0 if not found
    damage_state = determine_tank_damage_state(pga_value)
    operational_level = tank_operational_levels.get(damage_state, 1.0)
    # Adjust tank's initial water level based on operational level
    tank.init_level = tank.init_level * operational_level  

### 3-4- Evaluating Earthquake Effects on Pumps

For pumps, serviceability is determined by their operational capacity post-earthquake compared to their original capacity then adjusting capacities to 90%, 80%, 60%, or 0% for minor, moderate, extensive, or complete damage, respectively. 

In this study, we consider only the physical damage to pumps, which is modeled using fragility curves.

Similar to tanks, we employ fragility curves developed by HAZUS-MH for pumps. These curves are also represented as lognormal distributions with PGA as the engineering demand parameter. The corresponding median and lognormal standard deviation (β) values for pumps are as follows:

| Damage State | Median (g) | β   |
|--------------|------------|-----|
| Minor        | 0.15       | 0.75|
| Moderate     | 0.36       | 0.65| 
| Extensive    | 0.77       | 0.65|
| Complete     | 1.50       | 0.80|

Similar to tanks, pump capacities are reduced to 60%, 80%, and 90% for extensive, moderate, and minor damage states, respectively.|


In [7]:
# Lognormal distribution parameters for each damage state (median, beta) of the pumps
pump_damage_state_params = {'Minor': (0.15, 0.75), 'Moderate': (0.36, 0.65), 'Extensive': (0.77, 0.65), 'Complete': (1.5, 0.8)}

# Define operational levels for each damage state of the pumps
pump_operational_levels = {'None': 1.0, 'Minor': 0.9, 'Moderate': 0.8, 'Extensive': 0.6, 'Complete': 0.0}

# Function to determine the damage state of a pump based on its PGA value
def determine_pump_damage_state(pga):
    probabilities = {}
    previous_probability = 0
    for state, params in pump_damage_state_params.items():
        median, beta = params
        exceedance_probability = 1 - lognorm.cdf(pga, s=beta, scale=median)
        probabilities[state] = exceedance_probability - previous_probability
        previous_probability = exceedance_probability

    random_number = random.uniform(0, 1)
    damage_state = 'None'
    cumulative_probability = 0
    for state, probability in probabilities.items():
        cumulative_probability += probability
        if random_number <= cumulative_probability:
            damage_state = state
            break
    return damage_state

# Calculate the distance from the epicenter to each pump
pumps_distances = pumps_coordinates.apply(lambda row: great_circle((row['latitude'], row['longitude']), epicenter).meters, axis=1)

# Calculate the pga values at the location of each pump
pumps_pga = earthquake.pga_attenuation_model(pumps_distances)

# Determine the damage state for each pump and adjust its base speed level
pump_performance = {}
for pump_name in wn.pump_name_list:
    pump = wn.get_link(pump_name)
    pga_value = pumps_pga.get(pump_name, 0)  # Get the PGA value for each pump, default to 0 if not found
    damage_state = determine_pump_damage_state(pga_value)
    operational_level = pump_operational_levels.get(damage_state, 1.0)
    # pump.base_speed = operational_level

    pump_performance[pump_name] = operational_level

### 3-5- Analyzing Seismic Vulnerability of Pipelines

Based on the American Lifelines Alliance (ALA) guidelines from 2001, the repair rate (RR) is a key indicator to quantify the potential impact of seismic events on the pipeline infrastructure. RR represents the expected number of repairs needed per unit length of pipeline per year due to seismic hazards. In this study, we utilized the WNTR package to estimate the RR for each pipeline, employing the following regression model:
$$RR = C \times 0.000241 \times PGV$$
Where:
- $C$ is the correction factor, maps pipe characteristics to weights.
- $PGV$ is the peak ground velocity (m/s).
- $RR$ is the estimated number of repairs per meter per year due to TGD.

After estimating the repair rate, we determine the probability of failure for each pipeline by considering that even one damage is a failure. The probability of failure is calculated as the complement of the probability of zero failures, which follows a Poisson distribution:
$$P_f = 1 - e^{-RR \times \text{Length}}$$
Where:
- $P_f$ is the probability of failure of the pipeline.
- $RR$ is the repair rate of the pipeline.
- $\text{Length}$ is the length of the pipeline.

If the pipeline is deemed to fail (i.e., a random number between 0 and 1 is less than $P_f$), we then predict the number of damages per unit length of the pipeline using a Poisson random number with a mean equal to the repair rate **(Su et al. 1987; Fragiadakis and Christodoulou 2014; Farahmandfar et al. 2016)**.

The number of damages ($N_{\text{damages}}$) should then be divided into leaks ($N_{LKS}$) and breaks ($N_{BKS}$). In general, 15–20% of those damages result in breaks, while the rest represent leaks **(Hwang and Shinozuka 1998)**.
$$ N_{LKS} = 0.85 \times N_{damages} $$
$$ N_{BKS} = 0.15 \times N_{damages} $$

Ultimately, to model the damage, we add an orifice with the following opening area to the damaged pipeline **(Farahmandfar and Piratla, 2017)**:
$$ A_T = (0.03 \times N_{LKS} + 0.2 \times N_{BKS}) \times A $$
Where $A_T$ is the total orifice area and $A$ is the pipeline cross-sectional area.R

In [8]:
# Calculate the distance from the epicenter to each pipeline
pipelines_distances = pipelines_coordinates.apply(lambda row: great_circle((row['latitude'], row['longitude']), epicenter).meters, axis=1)

# Calculate the pgv values at the location of each pipeline in m/s
pipelines_pgv = earthquake.pgv_attenuation_model(pipelines_distances)

# Calculate the RR values at the location of each pipeline in number of repairs per meter per year
RR = earthquake.repair_rate_model(pipelines_pgv, C=0.5)

# Get the length of the pipelines from EPANET in ft and convert it to m
link_length = pd.Series(wn.query_link_attribute('length', link_type=wntr.network.Pipe)) * 0.3048

# Predict the number of damages per meter using a Poisson distribution
num_damages = {int(pipe): poisson.rvs(mu=repair_rate * link_length.iloc[int(pipe)]) for pipe, repair_rate in RR.items()}

# Divide the num_damages into leaks (85%) and breaks (15%)
leaks_breaks = {int(pipe): (int(0.85 * damage), int(0.15 * damage)) for pipe, damage in num_damages.items()}

# Get the diameters of the pipelines from EPANET in m
diameters = {int(pipe_name): wn.get_link(pipe_name).diameter for pipe_name in wn.pipe_name_list}

# Calculate the cross-sectional area of each pipeline (A = π * (d/2)^2)
areas = {pipe_name: np.pi * (d / 2) ** 2 for pipe_name, d in diameters.items()}

# Calculate the total orifice area (m2) for each damaged pipeline
orifice_areas = {pipe: min((0.03 * leaks + 0.2 * breaks) * areas[pipe], areas[pipe]) for pipe, (leaks, breaks) in leaks_breaks.items()}

# Adding orifice areas to pipelines in WNTR
for pipe, orifice_area in orifice_areas.items():
    # Calculate the probability of failure for the pipeline
    pf = 1 - np.exp(-RR[int(pipe)] * link_length.iloc[int(pipe)])
    
    # Check if the pipeline fails
    if np.random.uniform() < pf:
        # Split the pipe and add a leak with the calculated orifice area
        wn = wntr.morph.split_pipe(wn, str(pipe), f"{pipe}_A", f"Leak_{pipe}")
        leak_node = wn.get_node(f"Leak_{pipe}")
        leak_node.add_leak(wn, area=orifice_area, start_time=leak_start_time, end_time=leak_end_time)

### 3-6- Running Hydraulic Simultion for the Damaged WDS

In this section, we perform a hydraulic simulation of the WDS using the WNTRSimulator. The simulation results are then used to calculate the serviceability of different components in the WDS.

Serviceability is a measure of how well the system is able to meet the required demands after an event such as an earthquake. It is calculated differently for tanks, junctions, and pumps:

- **Tanks:** Serviceability is defined as the ratio of the new average tank level to the original tank level. This indicates how much water is available in the tank compared to its initial level.

- **Junctions:** Serviceability is the ratio of the average satisfied demand to the required demand at each junction. This shows how well the junctions are able to meet the water demand.

- **Pumps:** Serviceability is based on the operational status of the pumps. It is a binary value indicating whether the pump is operational (1) or not (0).

In [9]:
# Run a hydraulic simulation
sim = wntr.sim.WNTRSimulator(wn)
results = sim.run_sim()

# Calculate serviceability
serviceability = {}
for node_name in wn.node_name_list:
    if node_name in wn.tank_name_list:
        # For tanks, serviceability is the ratio of new tank level to original tank level
        new_tanks_level = results.node['head'].loc[:, node_name].mean() - wn.get_node(node_name).elevation
        serviceability[node_name] = new_tanks_level / original_tank_levels[node_name]
    elif node_name in wn.junction_name_list:
        # For junctions, serviceability is the ratio of satisfied demand to required demand        
        satisfied_demand = results.node['demand'].loc[:, node_name].mean()
        required_demand = wntr.metrics.expected_demand(wn)[node_name].mean()
        if required_demand == 0:
            # Skip the node if the required demand is zero
            continue
        serviceability[node_name] = satisfied_demand / required_demand

for pump_name in wn.pump_name_list:
    # For pumps, serviceability is based on their operational level
    serviceability[pump_name] = pump_performance[pump_name] 

# Print serviceability of nodes
print("Serviceability of Nodes:")
for node, value in serviceability.items():
    if isinstance(value, pd.Series):
        # If value is a Series, take the first element
        value = value.iloc[0]
    print(f"{node}: {value:.2f}")

/opt/conda/lib/python3.11/site-packages/wntr/sim/hydraulics.py:274: UserWarning: Pump P1 has exceeded its maximum flow.
  warnings.warn('Pump ' + name + ' has exceeded its maximum flow.')
/opt/conda/lib/python3.11/site-packages/wntr/sim/hydraulics.py:274: UserWarning: Pump P2 has exceeded its maximum flow.
  warnings.warn('Pump ' + name + ' has exceeded its maximum flow.')
/opt/conda/lib/python3.11/site-packages/wntr/sim/hydraulics.py:274: UserWarning: Pump P3 has exceeded its maximum flow.
  warnings.warn('Pump ' + name + ' has exceeded its maximum flow.')
/opt/conda/lib/python3.11/site-packages/wntr/sim/hydraulics.py:274: UserWarning: Pump P5 has exceeded its maximum flow.
  warnings.warn('Pump ' + name + ' has exceeded its maximum flow.')
/opt/conda/lib/python3.11/site-packages/wntr/sim/hydraulics.py:274: UserWarning: Pump P6 has exceeded its maximum flow.
  warnings.warn('Pump ' + name + ' has exceeded its maximum flow.')
/opt/conda/lib/python3.11/site-packages/wntr/sim/hydraulics.

Serviceability of Nodes:
16: 0.36
17: 0.37
18: 0.10
19: 0.04
20: 0.06
21: 0.46
22: 0.04
23: 0.04
24: 0.08
25: 0.04
26: 0.08
27: 0.04
28: 0.08
29: 0.04
30: 0.08
31: 0.05
32: 0.08
33: 0.04
34: 0.04
35: 0.07
36: 0.08
37: 0.05
38: 0.16
39: 0.05
40: 0.05
41: 0.16
42: 0.16
43: 0.12
44: 0.12
45: 0.11
46: 0.12
47: 0.12
48: 0.12
49: 0.16
T1: 0.52
T2: 0.53
T3: 0.50
T4: 0.75
T5: 0.59
T6: 0.42
P1: 0.00
P2: 0.80
P3: 0.60
P4: 0.80
P5: 0.00
P6: 1.00
P7: 0.90
P8: 0.00
P9: 0.80
